<h2>Q1</h2>

![Alt text](q11.png)

<h2>Q2</h2>

In [170]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType
from pyspark import SparkConf
import feedparser
import findspark
findspark.init()
findspark.find()
conf = SparkConf().set("spark.executor.memory", "4g").set("spark.driver.memory", "4g")
spark = SparkSession.builder.master("local[*]").config(conf=conf).getOrCreate()

In [171]:
url = "https://news.google.com/rss/search?q=technology&hl=en-US&gl=US&ceid=US:en"
# The use of feedparser is from GPT4
feed = feedparser.parse(url)

news_items = []
for entry in feed.entries:
    news_items.append((
        entry.title,
        entry.link,
        entry.published,
        entry.description,
        entry.source.title if 'source' in entry else "null"
    ))
# news_items

In [172]:
columns = "title STRING, link STRING, pubDate STRING, description STRING, source STRING"
test_df = spark.createDataFrame(news_items, columns)

test_df.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                link|             pubDate|         description|              source|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Gartner: The Top ...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|        TechRepublic|
|How the new iPhon...|https://news.goog...|Mon, 09 Sep 2024 ...|<a href="https://...|            CBS News|
|ESR Unveils Walle...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|              Forbes|
|Amazon adding AI ...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|            The Hill|
|2024 35 Innovator...|https://news.goog...|Mon, 09 Sep 2024 ...|<a href="https://...|MIT Technology Re...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [173]:
# update the options with your db user and password

db_properties={}
db_properties['username']="postgres"
db_properties['password']="88062601"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['table']="google_news"
db_properties['driver']="org.postgresql.Driver"

test_df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

![Alt text](q2.png)

<h2>Q3</h2>

In [174]:
from pyspark.sql import functions as F
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY") #There will be 
def func3(df):
    df = df.withColumn("new",F.to_timestamp("pubdate", "EEE, dd MMM yyyy HH:mm:ss Z"))
    last_24_hours = F.current_timestamp() - F.expr('INTERVAL 1 DAY')
    result = df.filter((df["new"] >= last_24_hours))
    result = result.select("title", "link", "pubDate", "description", "source")
    return result


In [175]:
result_df = func3(test_df)
result_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                link|             pubDate|         description|              source|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Amazon adding AI ...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|            The Hill|
|Airbus CEO upbeat...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|             Reuters|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|             Digiday|
|China takes lead ...|https://news.goog...|Wed, 11 Sep 2024 ...|<a href="https://...|            VOA Asia|
|David Hone interv...|https://news.goog...|Tue, 10 Sep 2024 ...|<a href="https://...|       New Scientist|
|Moth aims to brin...|https://news.goog...|Wed, 11 Sep 2024 ...|<a href="https://...|         VentureBeat|
|Top 10: Technolog...|https://news.go

![Alt text](q3.png)

<h2>Q4</h2>

In [176]:
url = "https://news.google.com/rss/search?q=technology&hl=en-US&gl=US&ceid=US:en"
# The use of feedparser is from GPT4
feed = feedparser.parse(url)

news_items = []
for entry in feed.entries:
    news_items.append((
        entry.title,
        entry.link,
        entry.published,
        entry.description,
        entry.source.title if 'source' in entry else "null",
        "technology"
    ))

url = "https://news.google.com/rss/search?q=business&hl=enUS&gl=US&ceid=US:en"
feed = feedparser.parse(url)
for entry in feed.entries:
    news_items.append((
        entry.title,
        entry.link,
        entry.published,
        entry.description,
        entry.source.title if 'source' in entry else "null",
        "business"
    ))

url = "https://news.google.com/rss/search?q=sports&hl=enUS&gl=US&ceid=US:en"
feed = feedparser.parse(url)
for entry in feed.entries:
    news_items.append((
        entry.title,
        entry.link,
        entry.published,
        entry.description,
        entry.source.title if 'source' in entry else "null",
        "sports"
    ))

columns = "title STRING, link STRING, pubDate STRING, description STRING, source STRING, catagory STRING"
test_df = spark.createDataFrame(news_items, columns)

# test_df.show()
test_df.groupby("catagory").count().show()

+----------+-----+
|  catagory|count|
+----------+-----+
|technology|  102|
|  business|  100|
|    sports|  102|
+----------+-----+



In [177]:
# update the options with your db user and password

db_properties={}
db_properties['username']="postgres"
db_properties['password']="88062601"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['table']="google_news"
db_properties['driver']="org.postgresql.Driver"

test_df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

<h2>Q5</h2>

In [178]:
df_read = spark.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

In [179]:
df_read.groupby("catagory").count().show()

+----------+-----+
|  catagory|count|
+----------+-----+
|technology|  102|
|    sports|  102|
|  business|  100|
+----------+-----+



In [180]:
filter_df = df_read.filter(~F.lower(F.col("title")).contains("nfl"))
filter_df.groupby("catagory").count().show()

+----------+-----+
|  catagory|count|
+----------+-----+
|technology|  101|
|    sports|   89|
|  business|  100|
+----------+-----+



<h2>Q6</h2>

![Alt text](q6.png)